In [3]:
import pandas as pd
import boto3
import psycopg2
from io import StringIO

# Configurações do PostgreSQL no RDS
db_config = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'darlingityler11',
    'host': 'susep2.cvosq0muixiq.us-east-1.rds.amazonaws.com',
    'port': 5432
}

# Configurações do S3
bucket_name = 'handson-mackenzie'  # Nome do bucket
s3_file_key = 'trusted/postgres_data.csv'  # Caminho dentro do bucket

# Query para selecionar os dados
query = "SELECT * FROM SES_SEGUROS_WIDE;"

def fetch_postgres_data():
    try:
        # Conecta ao banco PostgreSQL
        conn = psycopg2.connect(**db_config)
        print("Conexão com PostgreSQL estabelecida.")

        # Lê os dados em um DataFrame
        df = pd.read_sql_query(query, conn)
        print("Dados extraídos com sucesso.")
        return df

    except Exception as e:
        print(f"Erro ao conectar ao PostgreSQL: {e}")
    finally:
        conn.close()

def upload_to_s3(dataframe, bucket, key):
    try:
        # Converte DataFrame para CSV em memória
        csv_buffer = StringIO()
        dataframe.to_csv(csv_buffer, index=False)

        # Inicializa cliente do S3
        s3 = boto3.client('s3')
        s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
        print(f"Arquivo enviado para S3: s3://{bucket}/{key}")

    except Exception as e:
        print(f"Erro ao fazer upload para o S3: {e}")

if __name__ == "__main__":
    # Extrai os dados do PostgreSQL
    df = fetch_postgres_data()
    
    if df is not None and not df.empty:
        # Faz upload para o S3
        upload_to_s3(df, bucket_name, s3_file_key)
    else:
        print("Nenhum dado foi encontrado para exportar.")


Conexão com PostgreSQL estabelecida.


/var/folders/3s/w88srlwx24sf_qfk5qzh711c0000gn/T/ipykernel_53098/4061537114.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Dados extraídos com sucesso.
Arquivo enviado para S3: s3://handson-mackenzie/trusted/postgres_data.csv
